---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_towns = pd.read_table('university_towns.txt',header = None)
    state_names = university_towns.loc[university_towns[0].str.contains('edit')]
    state_ind = state_names[0].str.extract('([A-Z]\w{0,})', expand=True).index.values.tolist()

    lst=[]
    state=''
    region=''

    with open('university_towns.txt') as file:
        for line in file:
            if '[edit]' in line:
                state = line.split("[")[0]
                state = state.strip(' \n\t')
            else:
                region = line.split("(")[0]
                region = region.strip(' \n\t')
            #lst = [state,region]
            lst.append([state,region])

    ans = pd.DataFrame(lst,columns = ['State','RegionName'])
    ans = ans.drop(state_ind).reset_index(drop=True)
    return (ans)
get_list_of_university_towns()


In [10]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',header = None)
    gdp = gdp[[4,6]]
    gdp.columns=['Quarter','GDP']
    ind = gdp[gdp['Quarter']=='2000q1'].index.values.astype(int)[0]
    gdp = gdp.iloc[ind:].reset_index(drop=True)
    gdp['Change'] = np.where(gdp['GDP'].diff() > 0, 'increase', 'decline')
    df = pd.DataFrame()
    for i in range(len(gdp)):
        if((gdp.iloc[i][2] == 'decline')and(gdp.iloc[i+1][2] == 'decline')):
            df=df.append(gdp.iloc[i],ignore_index=True)
    ans = df.iloc[0]['Quarter']
    
    return (ans)
get_recession_start()

'2008q3'

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',header = None)
    gdp = gdp[[4,6]]
    gdp.columns=['Quarter','GDP']
    ind = gdp[gdp['Quarter']=='2000q1'].index.values.astype(int)[0]
    gdp = gdp.iloc[ind:].reset_index(drop=True)
    gdp['Change'] = np.where(gdp['GDP'].diff() > 0, 'increase', 'decline')
    df = pd.DataFrame()
    for i in range(len(gdp)):
        if((gdp.iloc[i][2] == 'decline')and(gdp.iloc[i+1][2] == 'decline')and(gdp.iloc[i+2][2] == 'increase')and(gdp.iloc[i+3][2] == 'increase')):
            df=df.append(gdp.iloc[i+3],ignore_index=True)
    ans = df.iloc[0]['Quarter']
    
    return (ans)       
get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',header = None)
    gdp = gdp[[4,6]]
    gdp.columns=['Quarter','GDP']
    ind = gdp[gdp['Quarter']=='2000q1'].index.values.astype(int)[0]
    gdp = gdp.iloc[ind:].reset_index(drop=True)
    gdp['Change'] = np.where(gdp['GDP'].diff() > 0, 'increase', 'decline')
    q1 = get_recession_start()
    q2 = get_recession_end()
    df = gdp[gdp.Quarter.between(q1,q2,inclusive=True)].sort(['GDP'],ascending=True)
    ans = df.iloc[0]['Quarter']
    return (ans)
get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    import datetime
    import numpy
    
    city = pd.read_csv('City_Zhvi_AllHomes.csv')
    city = city.drop(city.loc[:,'Metro':'1999-12'].columns, axis = 1) 
    del city['RegionID']
    df_index = city.iloc[:, :2]
    df = city.iloc[:, 2:]
    df.columns = pd.to_datetime(df.columns)
    res = df.resample('Q', axis=1).mean()
    res = res.rename(columns=lambda col: '{}q{}'.format(col.year, col.quarter))
    ans = df_index.join(res)
    ans = ans.replace({'State':states})
    ans.set_index(['State','RegionName'], inplace=True, drop=True)    
    return (ans)
convert_housing_data_to_quarters()


In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    import numpy as np
    import pandas as pd

    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    univ_towns = get_list_of_university_towns()
    housing = convert_housing_data_to_quarters()
    before_rec = housing.columns[housing.columns.get_loc(rec_start)-1]
    housing = housing.loc[:,[before_rec,rec_bottom]].reset_index().dropna()
    housing['ratio'] = housing[before_rec]/housing[rec_bottom]

    ar = np.intersect1d(housing[['RegionName']], univ_towns[['RegionName']])
    univ_towns_index = housing[housing['RegionName'].isin(ar)].index.values.tolist()
    housing_index = housing.index.values.tolist()
    non_univ_towns_index = set(housing_index) - set(univ_towns_index)
    non_univ_towns_index = sorted(non_univ_towns_index)
    univ_towns = housing.loc[univ_towns_index,'ratio']
    non_univ_towns = housing.loc[non_univ_towns_index,'ratio']

    s,p = ttest_ind(univ_towns, non_univ_towns)
    different = True if p<0.01 else False
    better = "university town" if univ_towns.mean() < non_univ_towns.mean() else "non-university town"

    return (different,p,better) 

run_ttest()